In [ ]:
cd ..

In [ ]:
from algorithms import np
import pandas as pd
import itertools
import math
import functools
import numpy
import seaborn as sns
import collections
sns.set_style("white", {
        "font.family": "serif",
        "font.serif": ["Times", "Palatino", "serif"]
})
import matplotlib.pyplot as plt
%matplotlib inline
all_data = pd.read_csv('notebook_analysis/intact_whitenoise_1_100Hz.csv')
coorddata = pd.read_csv('notebook_analysis/node_coords.csv',
                        dtype={'Node':numpy.int64},index_col='Node')

def translater(nodeax):
    node, ax = int(nodeax[0:-1]),nodeax[-1]
    translat = coorddata['Index'].to_dict()
    return translat[node]+ax
nodes = [63,41,15,95,96,53,55,31,22,17,18,56,58,61,64,43,46,49,68,32,37,39,40,69]
allnodes = [i for i in coorddata.index.tolist() if i <100]
print(allnodes)
neighborhoods = {node:[node%4] for node in allnodes}
biggest_modx = lambda lst, x: max([i for i in lst if i%4==x])
largesteach = [(i,biggest_modx(allnodes, i)) for i in range(4)]
for i,largest in largesteach:
    if i+1 in range(4):
        neighborhoods[largest].append(i+1)
print(neighborhoods)

In [ ]:
fig, ax= plt.subplots(1)
def get_fs(data, f):
    #eg f= 100 hz, duration =60s
    nbins = int(len(data)/2)
    nyquist = f/2
    stepsize = nyquist/nbins
    fftbins = [i*stepsize for i in range(nbins)]
    return fftbins
def maxer(fs,response):
    biggest = sorted(response,key = lambda x: abs(x), reverse=True)[:10]
    def getvals(x):
        idx = response.index(x)
        return idx, fs[idx]
    return [getvals(i) for i in biggest]
data = all_data['41x'].tolist()[0:2**12]
fft = list(map(abs, np.fft(data)))
fs = get_fs(data, 100)
ax.plot(fs, fft[0:2048], color = '#737373', linewidth=1)
ax.set_xlabel('Frequency (Hz)')
ax.set_xlim([0,50])
ax.set_ylim([10**-3, 10**0])
ax.set_yscale('log')
sns.despine()
#print('largest response at {}Hz, index{}'.format(*maxer(fs, fft[0:2048])))
maxer(fs, fft[0:2048])

In [ ]:
def samplestream(nodes, all_data):
    all_idxs = (str(x)+y for x,y in itertools.product(nodes, ['x','y','z']))
    return [(x, all_data[translater(x)].tolist()[0:2**12]) for x in all_idxs]
def mapper(d):
    idx = 616
    nodeid, data = d
    ftpeak = np.fft(data)[idx]
    c = lambda d: (round(d.real,6),round(d.imag,6))
    return [(0,(nodeid,c(ftpeak)))]
def shuffler(kvstream):
    flat = list(itertools.chain(*kvstream))
    sortedkvs = sorted(flat, key=lambda x: x[0])
    grouped = itertools.groupby(sortedkvs, key=lambda x:x[0])
    return ((k,[i[1] for i in g]) for k,g in grouped)
def reducer(kvs):
    k,vs =kvs
    ws = [complex(*i[1]) for i in vs]
    G = np.spectral_mat(ws)
    eig = np.pagerank(G)
    c = lambda d: (round(d.real,2),round(d.imag,2))
    ms = [(vs[idx][0],c(el)) for idx,el in enumerate(eig)]
    return (k,ms)
def dmapper(neighborhoods, d):
    nodeid, data = d
    nodenum, nodeax = int(nodeid[0:-1]), nodeid[-1]
    hoods = neighborhoods[nodenum]
    idx = 616
    ftpeak = np.fft(data)[idx]
    c = lambda d: (round(d.real,6),round(d.imag,6))
    hoods = hoods if nodeax=='x' else [hoods[0]]
    return [(i,(nodeid, c(ftpeak))) for i in hoods]

def mr(streamfun, mapfun, redfun):
    mappit = functools.partial(map, mapfun)
    reducit = functools.partial(map,redfun)
    return list(reducit(shuffler(mappit(streamfun()))))
def merger(lst):
    accer = lambda acc, el: acc+el[1]
    return functools.reduce(accer, lst,[])
def dfdd_merger(lst):
    """for example [('0',[('a',(1,2)),('b',(3,4))],
                    ('1',[('b',(2,3)),('c',(4,5))]) -? to equalise bs, must subtract (1,1) from everything in 2
                    ]
    should return [('a',(1,2)),('b',(3,4)),('c',(3+4))
    """
    def accer(acc, el):
        def tuplediff(x,y):
            return (x[0], (x[1][0]-y[1][0],x[1][1]-y[1][1]))
        def tupleadd(x,y):
            return (x[0], (x[1][0]+y[1][0],x[1][1]+y[1][1]))
        def match_or_none(lst, key):
            try:
                matcher = next((i for i in allcombs if key(i)))
                return matcher
            except StopIteration:
                return 
        newnodes = el[1]
        allcombs = list(itertools.product(acc, newnodes))
        matcher = match_or_none(allcombs, key=lambda i: i[0][0]==i[1][0])
        diff = tuplediff(*matcher) if matcher else ('None',(0,0))
        print('diff: ', diff)
        superposed = [tupleadd(i,diff) for i in newnodes]
        acc.update(superposed)
        return acc
    return functools.reduce(accer, lst, set())
import functools
    #return functools.reduce(accer, lst, set())
dfdd_merger([('0',[('a',(1,2)),('b',(3,4))]),
             ['1',[('b',(2,3)),('c',(4,5))]],
             ['2',[('c',(1,3)),('e',(9,1))]]
           ])

In [ ]:
stream = functools.partial(samplestream,allnodes, all_data)
r = mr(stream, mapper, reducer)
res = merger(r)

In [ ]:
streamdfdd = functools.partial(samplestream, allnodes, all_data)
dfddmapper=functools.partial(dmapper,neighborhoods)
rdfdd = mr(streamdfdd, dfddmapper, reducer)
resdfdd = dfdd_merger(rdfdd)

In [ ]:
def realify(amplitude, phase):
    """take complex mode shape, which means we can express structural
    motion at each point as something like a*sin(wt+phi),
    and return a real value which is a snapshot of this steady state
    motion at t=0"""
    t=0
    w=_
    print('a, phi',amplitude, phase)
    return abs(complex(amplitude, phase))#amplitude*math.sin(phase)
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z
def mergeaxes(entries):
    """entries is a list of modeshape values for
    a specific node,for example:
    [{'ax': 'x', 'Node': '31', 'val': -0.03}
    , {'ax': 'y', 'Node': '31', 'val': 0.05}]"""
    convert = lambda d: {'Node':d['Node'],2*d['ax']:d['val']}
    return functools.reduce(merge_two_dicts, map(convert, entries),{})
def formatter(res):
    d = [{'Node':k[0:-1],'ax':k[-1], 'val':realify(*v)} for k,v in res]
    print('d: ',d)
    sortedd = sorted(d, key=lambda d:d['Node'])
    grouped = itertools.groupby(sortedd, key = lambda d:d['Node'])
    res = [mergeaxes(group) for k, group in grouped]
    print('res: ',res)
    return pd.DataFrame(res)


In [ ]:
final = formatter(res)
final.to_csv('notebook_analysis/fddmodeshape.csv', index=False)

In [ ]:
dfinal = formatter(resdfdd)
dfinal.to_csv('notebook_analysis/dfddmodeshape.csv', index=False)